<a href="https://colab.research.google.com/github/leenago/NLP_edu/blob/master/200908_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D__%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes 감정분류 by using sklearn

## 1. Google Drive에 Mount하기

In [1]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)   

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/bigdata/아경/감성분석

/content/drive/My Drive/bigdata/아경/감성분석


## 2. 데이터 정제하기

In [3]:
import pandas as pd
import numpy as np
df = pd.DataFrame(pd.read_csv('ratings_train.txt', sep= '\t', quoting = 3))
#quoting -> 열이 3개라는 뜻, 
df = df.replace(np.nan,'', regex = True)
#non값은 ''(빈 공간으로 채워주라는 뜻) 처리를 함. (drop.na 하지 않음)
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
import re #정규표현식을 하기 위한 처리

#한글을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")
#compile을 한 번 해주면 속도가 더 빨라진다.

def preprocess(text):
  text = re.sub(remove_except_ko,' ', text).strip()
  #re.sub = replace / ' ' 공백처리 / input으로 들어오는 text를, strip은 좌우 공백 없애게 해달라는 뜻
  return text

df['document'] = df['document'].map(lambda x : preprocess(x))
#lambda x (document)
#map.lamda를 하면 위에서 부터 찍어서 내려오기 때문에, for문을 사용하는 것보다 속도가 빠르다.

df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## 3. Konlpy-mecab 토큰화
- 불용어 처리, 한 글자 제거하는 작업

In [5]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 11.5MB/s 
     |████████████████████████████████| 3.8MB 47.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [6]:
! sudo apt-get install g++ openjdk-7-jdk #Install Java 1.7+ 
!sudo apt-get install python-dev; pip install konlpy # Python 2.x 
#얘가 안깔려있어서 mecab이 안됐던 것인가..
!sudo apt-get install python3-dev; pip3 install konlpy # Python 3.x 
!sudo apt-get install curl 
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded,

In [7]:
from konlpy.tag import Mecab
mecab = Mecab()

#mecab을 사용해서 토큰화, 한글자 제거, stop_word 제거
stop_word = ['께서','에서','이다','에게','으로','이랑','까지','부터','하다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX','SF',
           'SE','SS','SP','SSO','SSC','SC','SY','SJ','SL','SN',
           'MM','MAG','MAJ']

def token_mecab(text):
  text = mecab.pos(text)
  #mecab에 pos tagging -> 단어가 0
  text = [i for i in text if len(i[0])>1] #단어 길이가 1보다 긴 
  text = [i for i in text if i[0] not in stop_word] 
  text = [i[0] for i in text if i[1] not in stopPos] 
  return text

df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
#토큰화 & 토큰리스트 생성
def make_tokens(df):
  df['tokens'] = '' #df DataFrame에 'token' 이라는 열을 만들고, 그 안에 아무것도 채우지 않는 것!

  for i, row in df.iterrows():
    if i%10000 ==0:
      print(i,'/', len(df))  #진행 과정을 보여주는!
    token = token_mecab(df['document'][i]) #document를 쪼개서 mecab처리
    df['tokens'][i] = ' '.join(token) #토큰 사이를 공백으로 처리
  return df

df = make_tokens(df) #바로 위에서 만든 함수

0 / 150000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


10000 / 150000
20000 / 150000
30000 / 150000
40000 / 150000
50000 / 150000
60000 / 150000
70000 / 150000
80000 / 150000
90000 / 150000
100000 / 150000
110000 / 150000
120000 / 150000
130000 / 150000
140000 / 150000


In [10]:
df.head()

,id,document,label,tokens
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,짜증 네요 목소리
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,포스터 초딩 영화 오버 연기 가볍 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,교도소 이야기 구먼 재미 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그 익살 스런 연기 돋보였 영화 스파이더맨 보이 커스틴 던스트 이뻐 보였


## 4. RNN 학습- 예측 결과 정확도 측정


In [12]:
X_train = list(df['tokens'])
Y_train = list(df['label'])

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#CountVectorizer 선언
count_vect = CountVectorizer()

#fit and transform
X_train_counts = count_vect.fit_transform(X_train)

#MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [15]:
test = '전 세계가 주목하는 올해의 영화답다. 완벽했다.'
test = token_mecab(test)
test = ' '.join(test)
print(test)

#예측
print(clf.predict(count_vect.transform([test])))
print(clf.predict_proba(count_vect.transform([test])))

세계 주목 올해 영화 완벽
[1]
[[0.06329792 0.93670208]]


## 5. RNN 학습모델로 test 데이터 예측

In [19]:
import pandas as pd
test_df = pd.DataFrame(pd.read_csv('ratings_test.txt', sep = '\t', quoting = 3))
test_df = test_df.replace(np.nan,'',regex = True)
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [20]:
test_df['document'] = test_df['document'].map(lambda x : preprocess(x))
test_df = make_tokens(test_df) #위에서 만든 함수
test_df.head()

0 / 50000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


10000 / 50000
20000 / 50000
30000 / 50000
40000 / 50000


,id,document,label,tokens
0,6270596,굳 ㅋ,1,
1,9274899,,0,
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편


In [24]:
predic_label = []
for i, row in test_df.iterrows():
  predic = clf.predict(count_vect.transform([test_df['tokens'][i]]))[0]
  #[0]을 해주지 않으면, 결과값이 [0]이 된다.
  #clf = MultinomialNB().fit(X_train_counts, Y_train)
  predic_label.append(predic)

test_df['predic_label'] = predic_label
test_df.head()

,id,document,label,tokens,predic_label
0,6270596,굳 ㅋ,1,,0
1,9274899,,0,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,평점 나쁘 지만 짜리 아니 잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 은데 완전 막장,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,아니 어도 다섯 텐데 나와서 심기 불편,0


## 6. Confusion metrix

- 예측 참/결과 참: TP
- 예측 거짓 / 결과 거짓: TN
- 예측 거짓 / 결과 참: FT
- 예측 참/ 결과 거짓: FN

In [27]:
TP = FN = FP = TN = other = 0
a = test_df['label']
b = test_df['predic_label']

for i in range(len(test_df)):
  if a[i] == b[i] == 1:
    TP += 1
  elif a[i] == b[i] == 0:
    TN += 1
  elif a[i] == 1 and b[i]==0:
    FN += 1
  elif a[i]==0 and b[i]==1:
    FP += 1
  else:
    other +=1
print('TP: ',TP, ' TN: ', TN, ' FP: ', FP,  ' FN: ', FN) 

TP:  19632  TN:  20393  FP:  4434  FN:  5541


In [28]:
ACC = (TP+TN)/(TP+TN+FN+FP)
print('ACC: ', ACC)

ACC:  0.8005
